# Домашнее задание к лекции "Основы веб-скрапинга"

## Обязательная часть

Вам необходимо написать функцию, которая будет основана на **поиске** по сайту [habr.com](https://habr.com/ru/search/).
Функция в качестве параметра должна принимать **список** запросов для поиска (например, `['python', 'анализ данных']`) и на основе материалов, попавших в результаты поиска по **каждому** запросу, возвращать датафрейм вида:

```
<дата> - <заголовок> - <ссылка на материал>
```

В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.


## Дополнительная часть (необязательная)

Функция из обязательной части задания должна быть расширена следующим образом:
- кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента `4` необходимо получить материалы с первых 4 страниц результатов;
- в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:
```
<дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>
```


In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from pandas.core.common import flatten

In [2]:
url = 'https://habr.com'

In [3]:
def page_url(page):
    """Возвращает url страницы с номером page"""
    return url + f'/ru/search/page{page}/'

In [4]:
def get_habr_page_link(query, page):
    """Возвращает ссылки со страницы по заданному запросу"""
    res = requests.get(
        page_url(page),
        params={'q': query}
    )
    if res.status_code != 200:
        return {}
    links = BeautifulSoup(res.text).find_all('a', class_='tm-title__link')
    return {link['href'] for link in links}

In [5]:
def get_query_links(query, pages):
    """Возвращает все ссылки с pages страниц по запросу query"""
    return set(flatten(
        [get_habr_page_link(query, page) for page in range(1, pages+1)]
        ))

In [26]:
def parse_page(link):
    """Парсит статью по конкретной ссылке"""
    page_url = url + link
    res = requests.get(page_url)
    if res.status_code != 200:
        return {
            'link': page_url,
            'title': '!!! ОШИБКА !!!',
            'text': 'Ошибка доступа к странице'
        }
    try:
        page = BeautifulSoup(res.text)
        title = page.find('h1', class_='tm-title tm-title_h1').span.text
        date = page.find('span', class_='tm-article-datetime-published').time['title']
        text = page.find('div', class_='article-formatted-body').text
        like = int(page.find('span', class_='tm-votes-meter__value').text)
    except Exception as err:
        return {
            'link': page_url,
            'title': '!!! ОШИБКА !!!',
            'text': 'Ошибка парсинга страницы\n' + str(err)
        }
    return {
        'link': page_url,
        'text': text,
        'title': title,
        'date': date,
        'like': like
    }
    

In [7]:
def get_pages_links(queries, pages):
    """Получает все ссылки по запросам queries глубиной pages страниц"""
    links = set()
    links.update(
        *[get_query_links(query, pages) for query in queries]
    )
    return links
    

In [8]:
def get_habr_pages(queries, pages):
    """Формирует необходимый DataFrame"""
    links = get_pages_links(queries, pages)
    df = pd.DataFrame()
    for link in links:
        df = pd.concat(
            [df, pd.DataFrame([parse_page(link)])]
            )
    return df

In [27]:
habr_df = get_habr_pages(['python', 'clojure'], 2)
habr_df

,link,text,title,date,like
0,https://habr.com/ru/articles/775440/,В наше время автоматизация стала ключевым факт...,Внедрение Автоматизации в Проект с Python: Шаг...,"2023-11-21, 17:13",2.0
0,https://habr.com/ru/articles/770216/,"Привет, хабр! Сегодня мы хотели бы продолжить ...",“Ну и долго мне ещё до магазина?” Или пара сло...,"2023-10-26, 20:57",10.0
0,https://habr.com/ru/articles/164701/,\n\r\nВ этом семестре (осенью 2012) был провед...,Clojure в Белорусском ГУ,"2013-01-05, 15:09",37.0
0,https://habr.com/ru/news/531402/,По данным широко известного в узких кругах Tio...,Python как компилируемый статически типизирова...,"2020-12-04, 21:03",16.0
0,https://habr.com/ru/articles/84204/,Предисловие\r\nНедавно я начал изучать язык Cl...,"Clojure и Project Euler, часть 1","2010-02-15, 18:00",31.0
...,...,...,...,...,...
0,https://habr.com/ru/articles/84791/,"В предыдущей статье (Clojure и Project Euler, ...","Clojure и Project Euler, часть 2","2010-02-22, 09:28",16.0
0,https://habr.com/ru/news/752190/,Привет! В рамках Moscow Python Podcast поделим...,Что нового в Python за июль — обсуждаем в прям...,"2023-08-03, 11:27",5.0
0,https://habr.com/ru/articles/222737/,\n\r\nНедавно я познакомился с интересным язык...,"Как я написал web-приложение, используя только...","2014-05-14, 14:30",48.0
0,https://habr.com/ru/companies/gazprommedia/art...,"Привет, Хабр! Меня зовут Катерина, я инженер п...","Make It Right! Максимум пользы, минимум пробле...","2023-11-27, 08:45",7.0


In [29]:
habr_df[habr_df['title'].str.contains('ОШИБКА')]

,link,text,title,date,like
0,https://habr.com/ru/companies/raiffeisenbank/n...,Ошибка парсинга страницы\n'NoneType' object ha...,!!! ОШИБКА !!!,NaN,NaN
0,https://habr.com/ru/articles/102197/,Ошибка доступа к странице,!!! ОШИБКА !!!,NaN,NaN
0,https://habr.com/ru/articles/104185/,Ошибка доступа к странице,!!! ОШИБКА !!!,NaN,NaN
0,https://habr.com/ru/companies/epam_systems/new...,Ошибка парсинга страницы\n'NoneType' object ha...,!!! ОШИБКА !!!,NaN,NaN
0,https://habr.com/ru/articles/137608/,Ошибка доступа к странице,!!! ОШИБКА !!!,NaN,NaN
